# Add CPC categories to a database 

In [1]:
from mescal import *
import pandas as pd
import bw2data as bd

In [2]:
bd.projects.set_current("ei3.8-mescal")

In [3]:
mapping_product_to_CPC = pd.read_csv('data/mapping_product_to_CPC.csv')

In [4]:
name_premise_db = "ecoinvent_cutoff_3.8_remind_SSP2-Base_2020"
premise_db = load_extract_db(name_premise_db)

In [5]:
name_premise_comp_db = name_premise_db + '_comp_QC'
premise_comp_db = load_extract_db(name_premise_comp_db)

## Create a new database with additional CPC categories

In [6]:
name_premise_with_CPC_db = name_premise_db+'_with_CPC'
name_premise_comp_with_CPC_db = name_premise_comp_db+'_with_CPC'

In [7]:
create_new_database_with_CPC_categories(db=premise_db, new_db_name=name_premise_with_CPC_db, mapping_product_to_CPC=mapping_product_to_CPC)

Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:35


Title: Writing activities to SQLite3 database:
  Started: 06/12/2024 16:45:04
  Finished: 06/12/2024 16:45:39
  Total time elapsed: 00:00:35
  CPU %: 3.50
  Memory %: 9.57


In [8]:
create_new_database_with_CPC_categories(db=premise_comp_db, new_db_name=name_premise_comp_with_CPC_db, mapping_product_to_CPC=mapping_product_to_CPC)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/12/2024 16:47:39
  Finished: 06/12/2024 16:47:39
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 10.74


Relink the complementary database with the newly created main database

In [9]:
premise_comp_db_with_CPC = load_extract_db(name_premise_comp_with_CPC_db, create_pickle=False)

Getting activity data


100%|██████████| 37/37 [00:00<?, ?it/s]


Adding exchange data to activities


100%|██████████| 1170/1170 [00:00<00:00, 37810.40it/s]


Filling out exchange data


100%|██████████| 37/37 [00:00<00:00, 192.18it/s]


In [10]:
relink_database(premise_comp_db_with_CPC, name_premise_db, name_premise_with_CPC_db)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/12/2024 16:47:39
  Finished: 06/12/2024 16:47:39
  Total time elapsed: 00:00:00
  CPU %: 10.00
  Memory %: 10.73


## Test the new database

In [11]:
premise_db_with_CPC = load_extract_db(name_premise_with_CPC_db)

Getting activity data


100%|██████████| 30201/30201 [00:00<00:00, 32914.38it/s]


Adding exchange data to activities


100%|██████████| 1022813/1022813 [00:36<00:00, 27991.89it/s]


Filling out exchange data


100%|██████████| 30201/30201 [00:02<00:00, 11326.89it/s]


ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_with_CPC.pickle created!


In [12]:
db = premise_db_with_CPC + premise_comp_db_with_CPC

In [13]:
premise_db_with_CPC_dict = {(a['name'], a['reference product'], a['location']): a for a in db}

In [14]:
# Relink your mapping with the premise database before running the following cells
mapping_linked_to_premise = pd.read_csv('energyscope_data/mapping_QC_linked.csv')

In [15]:
mapping_linked_to_premise

,Name,Type,Product,Activity,Location,Database
0,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020
1,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020
2,ALKALINE_ELECTROLYSIS_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020
3,ALKALINE_ELECTROLYSIS_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020
4,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020
...,...,...,...,...,...,...
632,URANIUM,Resource,"nuclear fuel element, for pressure water react...","market for nuclear fuel element, for pressure ...",GLO,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020
633,WASTE,Resource,municipal solid waste,market for municipal solid waste,CA-QC,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020
634,WET_BIOMASS,Resource,"wood chips, wet, measured as dry mass","market for wood chips, wet, measured as dry mass",CA-QC,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020
635,WOOD,Resource,"wood chips, dry, measured as dry mass","market for wood chips, dry, measured as dry mass",RoW,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020


In [16]:
for i in range(len(mapping_linked_to_premise)):
    activity = mapping_linked_to_premise.loc[i, 'Activity']
    product = mapping_linked_to_premise.loc[i, 'Product']
    location = mapping_linked_to_premise.loc[i, 'Location']
    
    act = premise_db_with_CPC_dict[(activity, product, location)]
    try:
        CPC = dict(act['classifications'])['CPC']  
    except KeyError:
        print(f'No CPC category for: {product} - {activity} - {location}')